In [ ]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from ipywidgets import Image
from IPython import display
import numpy as np
from scipy.spatial import Delaunay
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
%matplotlib widget

In [ ]:
w1data = pd.read_csv("nfl-big-data-bowl-2023/week1.csv")
players_background = pd.read_csv("BigDataBowl2023/data2023/players.csv", index_col='nflId')
madden_df = pd.read_csv("BigDataBowl2023/merged_df.csv")
median_df = pd.read_csv("BigDataBowl2023/all_pos_df.csv", index_col=0)

In [ ]:
def collect_game_info(weekdf, playId, saved=False):
    
    #slice df to single play
    playdf = weekdf.loc[weekdf["playId"] == playId]
    #reorder playdf so that they are in the right frame order
    playdf = playdf.sort_values(by = "frameId")
    if saved:
        playdf.to_csv(f'Play{playId}Frames.csv')
    return playdf

In [ ]:
def gaussian(x):
    return np.exp(-(x-1)**2/0.4)

In [ ]:
def find_best_player(weekdf,playId):
    play = collect_game_info(weekdf, playId)
    madden_df.loc[:, 'nflId'] = madden_df['nflId'].astype('int64')
    madden_df = madden_df.set_index('nflId')
    play_df = play.dropna()
    ball_df = play[play['team'] == 'football']
    play_df.loc[:, 'frameId'] = play_df['frameId'].astype('int')
    play_df.loc[:, 'nflId'] = play_df['nflId'].astype('int64')
    player_df = play_df.set_index(['nflId', 'frameId'])
    ball_df = ball_df.set_index(['frameId'])

        # Stuff for plotting
    num_frames = len(play_df['frameId'].unique())
        #self.num_players = len(self.play_df['jerseyNumber'].unique())
    teams = player_df['team'].unique()
    for frame in range(1,num_frames):
        idx = pd.IndexSlice
        player_frame_df = self.player_df.loc[idx[:, i+1], :].droplevel(-1)
        ball_frame_df = self.ball_df.loc[i+1].to_frame().transpose() # the hatred towards pandas I have in this instant is unquantifiable with words
        ball_frame_df['nflId'] = 'football'
        ball_frame_df.set_index('nflId', inplace=True)

        # Find the nflId of the QB (theres probably a better way to do this but idk it)
        for nflId, player in player_frame_df.iterrows():
            pos = self.players_background.loc[nflId]['officialPosition']
            if pos =='QB':
                qb_Id = nflId
                qb_team = player['team']
                qbx, qby = player[['x', 'y']]


        # Sort the players into attackers and defenders
        attackers = player_frame_df[player_frame_df['team']==qb_team]
        defenders = player_frame_df[player_frame_df['team']!=qb_team]

        receiver_ratings = {'nflId':[], 'receiver rating':[]}

        qb_stats = self.madden_df.loc[qb_Id]

        throw_score = qb_stats['Throw On The Run'] + qb_stats['Throw Under Pressure'] + qb_stats['Throw Power'] # !!!


    